<a href="https://colab.research.google.com/github/choysf/My-first-Python-projects/blob/master/week_04_part_02_build_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
---

<h1>Notebook: [ Week #04: Building your own RAG Bot ]</h1>

- Your objective in this notebook is create a RAG Bot that allow the users to interact with some notes from AI Champions Bootcamp.
- A convenient way to work on this notebook is to open the earlier Jupyter Notebook in `Topic 4`. Yes, the notebook with pre-populated code cells.
- You can refer to how a simple RAG Bot (or more like a RAG pipeline) is built
- You may extend the functionalities of the bot as you wish.
- Minimumly, you should have a simple RAG Bot like the one in the earlier `Topic 4` Jupyter Notebook


---
---

# Setup

In [1]:
!pip install openai
!pip install langchain
!pip install langchain-openai
!pip install langchain-experimental
!pip install langchain-chroma
!pip install pypdf
!pip install lolviz
!pip install chromadb
!pip install tqdm
!pip install tiktoken

# You may need to install other dependencies that you need for your project

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.9/393.9 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.3 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import os
import openai
from getpass import getpass

# Set up the OpenAI API key by setting the OPENAI_API_KEY environment variable
os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API Key")


Enter your OpenAI API Key··········


---

## Helper Functions

---

### Function for Generating Embedding

In [4]:
def get_embedding(input, model='text-embedding-3-small'):
    response = client.embeddings.create(
        input=input,
        model=model
    )
    return [x.embedding for x in response.data]

### Function for Text Generation

In [5]:
# This is the "Updated" helper function for calling LLM
def get_completion(prompt, model="gpt-4o-mini", temperature=0, top_p=1.0, max_tokens=256, n=1, json_output=False):
    if json_output == True:
      output_json_structure = {"type": "json_object"}
    else:
      output_json_structure = None

    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create( #originally was openai.chat.completions
        model=model,
        messages=messages,
        temperature=temperature,
        top_p=top_p,
        max_tokens=max_tokens,
        n=1,
        response_format=output_json_structure,
    )
    return response.choices[0].message.content

In [6]:
# This a "modified" helper function that we will discuss in this session
# Note that this function directly take in "messages" as the parameter.
def get_completion_by_messages(messages, model="gpt-4o-mini", temperature=0, top_p=1.0, max_tokens=1024, n=1):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        top_p=top_p,
        max_tokens=max_tokens,
        n=1
    )
    return response.choices[0].message.content

## Functions for Token Counting

In [7]:
# These functions are for calculating the tokens.
# ⚠️ These are simplified implementations that are good enough for a rough estimation.

import tiktoken

def count_tokens(text):
    encoding = tiktoken.encoding_for_model('gpt-4o-mini')
    return len(encoding.encode(text))

def count_tokens_from_message(messages):
    encoding = tiktoken.encoding_for_model('gpt-4o-mini')
    value = ' '.join([x.get('content') for x in messages])
    return len(encoding.encode(value))


---
---

# Create a "Chat with your Document" Bot

**\[ Overview of Steps in RAG \]**

- 1. **Document Loading**
	- In this initial step, relevant documents are ingested and prepared for further processing. This process typically occurs offline.
- 2. **Splitting & Chunking**
	- The text from the documents is split into smaller chunks or segments.
	- These chunks serve as the building blocks for subsequent stages.
- 3. **Storage**
	- The embeddings (vector representations) of these chunks are created and stored in a vector store.
	- These embeddings capture the semantic meaning of the text.
- 4. **Retrieval**
	- When an online query arrives, the system retrieves relevant chunks from the vector store based on the query.
	- This retrieval step ensures that the system identifies the most pertinent information.
- 5. **Output**
	- Finally, the retrieved chunks are used to generate a coherent response.
	- This output can be in the form of natural language text, summaries, or other relevant content.

![](https://abc-notes.data.tech.gov.sg/resources/img/topic-4-rag-overview.png)

---
---

## Document Loading

Here are the "notes" that you must include in your RAG pipeline as the `Documents`
- [Key Parameters for LLMs](https://abc-notes.data.tech.gov.sg/notes/topic-2-deeper-dive-into-llms/2.-key-parameters-for-llms.html)
- [LLMs and Hallucinations](https://abc-notes.data.tech.gov.sg/notes/topic-2-deeper-dive-into-llms/3.-llms-and-hallucinations.html)
- [Prompting Techniques for BUilders](https://abc-notes.data.tech.gov.sg/notes/topic-2-deeper-dive-into-llms/4.-prompting-techniques-for-builders.html)

You have three options.
1) 💪🏼 Take up the challenge to find a way to get the content directly from the webpages above.
2) 🥴 Go with the easy route, download the notes nicely prepared in a `.txt` format. Download the zipped file [here](https://abc-notes.data.tech.gov.sg/resources/data/notes.zip)
3) 😎 “Only children choose; adults take all.” Experiment with both data sources and see which can help to the Bot to provide more accurate information for the user queries.

---

> 💡 **Feel free to add as many code cells as your need.**

---

In [11]:
# < Your Code Here >
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader
loader = DirectoryLoader('/content/sample_data/week4', glob="**/*.txt", loader_cls=TextLoader)
docs = loader.load()
print(type(docs))

<class 'list'>


## Splitting & Chunking

In [ ]:
# < Your Code Here >

## Storage: Embedding & Vectorstores

In [ ]:
# < Your Code Here >

## Retrieval

In [ ]:
# < Your Code Here >

## Question & Answer

In [ ]:
# < Your Code Here >